# Setup

### Imports

In [1]:
import gc
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

!pip install wandb -qqq
import wandb
# from wandb.keras import WandbCallback
wandb.login()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Loading in DataFrames

In [2]:
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

# Preprocessing

### Feature Engineering Function

In [3]:
def add_features(df):
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    df['cross']= df['u_in']*df['u_out']
    df['cross2']= df['time_step']*df['u_out']
    
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    df['u_in_cummean'] =df['u_in_cumsum'] /df['count']
    
    #df['u_in_lag']=0
    #df['u_in_lag2']=0
    #for i in range(df.shape[0]):
        #if df['breath_id'][i]==df['breath_id'][i+1]:
        #    df['u_in_lag'][i+1]=df['u_in'][i]
        #else:
        #    df['u_in_lag'][i+1]=0
        #if df['breath_id'][i]==df['breath_id'][i+2]:
        #    df['u_in_lag'][i+2]=df['u_in'][i]
        #else:
        #    df['u_in_lag'][i+2]=0
        #if i/10000==round(i/10000):
        #    print(i)
    
    df['breath_id_lag']=df['breath_id'].shift(1).fillna(0)
    df['breath_id_lag2']=df['breath_id'].shift(2).fillna(0)
    df['breath_id_lagsame']=np.select([df['breath_id_lag']==df['breath_id']],[1],0)
    df['breath_id_lag2same']=np.select([df['breath_id_lag2']==df['breath_id']],[1],0)
    df['u_in_lag'] = df['u_in'].shift(1).fillna(0)
    df['u_in_lag'] = df['u_in_lag']*df['breath_id_lagsame']
    df['u_in_lag2'] = df['u_in'].shift(2).fillna(0)
    df['u_in_lag2'] = df['u_in_lag2']*df['breath_id_lag2same']
    df['u_out_lag2'] = df['u_out'].shift(2).fillna(0)
    df['u_out_lag2'] = df['u_out_lag2']*df['breath_id_lag2same']
    #df['u_in_lag'] = df['u_in'].shift(2).fillna(0)
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['RC'] = df['R']+df['C']
    df = pd.get_dummies(df)
    return df

### Preprocessing the Dataset

In [39]:
# Feature Engineering.
train_ft = add_features(train)
test_ft = add_features(test)

# Filling missing values.
train_ft = train_ft.fillna(0)
test_ft = test_ft.fillna(0)

# Getting targets and dropping unimportant features.
targets = train_ft[['pressure']].to_numpy().reshape(-1, 80)
train_ft.drop(['pressure','id', 'breath_id','one','count','breath_id_lag','breath_id_lag2','breath_id_lagsame','breath_id_lag2same','u_out_lag2'], axis=1, inplace=True)

# Getting Test IDs and dropping unimportant features.
test_ids = test["id"]
test_ft = test_ft.drop(['id', 'breath_id','one','count','breath_id_lag','breath_id_lag2','breath_id_lagsame','breath_id_lag2same','u_out_lag2'], axis=1)

# # Scaling.
# RS = RobustScaler()
# train_ft = RS.fit_transform(train_ft)
# test_ft = RS.transform(test_ft)

# # Reshaping.
# train_ft = train_ft.reshape(-1, 80, train_ft.shape[-1])
# test_ft = test_ft.reshape(-1, 80, train_ft.shape[-1])
# test_ids = np.array(test_ids).reshape(50300, 80)

# Getting u_out values.
# train_u_out = train.u_out.values.reshape(-1, 80)
# test_u_out = test.u_out.values.reshape(-1, 80)

In [40]:
train_ft

In [34]:
RS = RobustScaler()
train_ft = RS.fit_transform(train_ft)

In [38]:
pd.Series(train_ft[:, 2]).value_counts()

# Creating the Dataset

In [ ]:
class GeneralDataset(Dataset):
    def __init__(self, df, mode="train", targets=None, test_ids=None):
        super(GeneralDataset, self).__init__()
        assert mode in ["train", "test"]
        self.mode = mode
        self.df = df
        self.targets = targets
        self.test_ids = test_ids
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if self.mode == "train":
            X = torch.FloatTensor(self.df[idx])
            X = torch.unsqueeze(X, 0)
            y = torch.FloatTensor(self.targets[idx])
            return X, y
        
        ids = self.test_ids[idx]
        X = torch.FloatTensor(self.df[idx])
        X = torch.unsqueeze(X, 0)
        return ids, X
    
class BiGRUDataset(Dataset):
    def __init__(self, df, u_out_values, mode="train", targets=None, test_ids=None):
        super(BiGRUDataset, self).__init__()
        assert mode in ["train", "test"]
        self.df = df
        self.u_out_values = u_out_values
        self.mode = mode
        self.targets = targets
        self.test_ids = test_ids
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if self.mode == "train":
            X = torch.FloatTensor(self.df[idx])
            y = torch.FloatTensor(self.targets[idx])
            u_out_values = self.u_out_values[idx]
            return X, y
        
        ids = self.test_ids[idx]
        X = torch.FloatTensor(self.df[idx])
        return ids, X

# Creating the Model

In [ ]:
def mish(x):
    return (x*torch.tanh(F.softplus(x)))

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_l1 = nn.Conv2d(in_channels=1, out_channels=24, padding=1, stride=1, kernel_size=3)
        self.conv_l2 = nn.Conv2d(in_channels=24, out_channels=48, padding=0, stride=2, kernel_size=3)
        # self.linear_l1 = nn.Linear(3744, 4192)
        self.linear_l1 = nn.Linear(22464, 4192)
        self.linear_l2 = nn.Linear(4192, 512)
        self.linear_l3 = nn.Linear(512, 512)
        self.output = nn.Linear(512, 80)

    def forward(self, x):  # [B, 1, 80, 25]
        x = F.relu(self.conv_l1(x))  # [B, 24, 80, 25]
        x = mish(self.conv_l2(x))  # [B, 48, 39, 12]
        x = x.reshape(x.shape[0], -1)  # [B, 22464]
        x = mish(self.linear_l1(x))  # [B, 4192]
        x = F.relu(self.linear_l2(x))  # [B, 512]
        x = F.relu(self.linear_l3(x))  # [B, 512]
        x = mish(self.output(x))  # [B, 80]
        return x

class BiGRUModel(nn.Module):
    def __init__(self, n_features, h_features=80):
        super(BiGRUModel, self).__init__()
        self.GRU_block1 = nn.GRU(n_features, 512, num_layers=2, bidirectional=True, batch_first=True)
        self.GRU_block2 = nn.GRU(1024, 256, num_layers=2, bidirectional=True, batch_first=True) 
        self.GRU_block3 = nn.GRU(512, 128, num_layers=2, bidirectional=True, batch_first=True) 
        self.GRU_block4 = nn.GRU(256, 64, num_layers=2, bidirectional=True, batch_first=True)
        self.GRU_block5 = nn.GRU(128, h_features, num_layers=2, bidirectional=True, batch_first=True) 
#         self.GRU_block6 = nn.GRU(128, 256, num_layers=2, bidirectional=True, batch_first=True) 
#         self.GRU_block7 = nn.GRU(256, 512, num_layers=2, bidirectional=True, batch_first=True) 
#         self.GRU_block8 = nn.GRU(512, , num_layers=2, bidirectional=True, batch_first=True) 
        
        self.fc1 = nn.Linear(h_features * 2, 32)
        self.fc2 = nn.Linear(32, 1)

    def forward(self, x):  # [B, 80, n_features]
        x, _ = self.GRU_block1(x)  # [B, 80, 1024]
        x = mish(x)
        x, _ = self.GRU_block2(x)  # [B, 80, 512]
        x = mish(x)
        x, _ = self.GRU_block3(x)  # [B, 80, 256]
        x = mish(x)
        x, _ = self.GRU_block4(x)  # [B, 80, 128]
        x = mish(x)
        x, _ = self.GRU_block5(x)  # [B, 80, 160]
        x = mish(x)
        
        x = F.relu(self.fc1(x))  # [B, 80, 32]
        x = F.relu(self.fc2(x))  # [B, 80, 1]
        
        x = torch.squeeze(x, -1)  # [B, 80]
        
        return x

# Training Script

In [ ]:
# run.finish()

### Hyperparameters for CNN

In [ ]:
ckpt_dir = "."
model_name = "CNN"
project_name = f"G-Vent-{model_name}"
n_features = train_ft.shape[-1]
n_splits = 5
n_epochs = 10
init_lr = 1e-3
b_size = 512
num_workers = 16

### CNN Training

In [ ]:
kf = KFold(n_splits=n_splits, shuffle=True, random_state=2021)

for fold, (train_idx, test_idx) in enumerate(kf.split(train_ft, targets)):
    
    CHECKPOINT = '{}/{}_{}.pth'.format(ckpt_dir, model_name, fold)
    
    run = wandb.init(project=project_name, name=f"fold{fold}")
    
    print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
    X_train, X_valid = train_ft[train_idx], train_ft[test_idx]
    y_train, y_valid = targets[train_idx], targets[test_idx]
    
    train_ds = GeneralDataset(X_train, mode="train", targets=y_train)
    valid_ds = GeneralDataset(X_valid, mode="train", targets=y_valid)
    
    train_loader = DataLoader(train_ds, batch_size=b_size, sampler=RandomSampler(train_ds), num_workers=num_workers)
    valid_loader = DataLoader(valid_ds, batch_size=b_size, num_workers=num_workers)
    
    model = CNN()
    
    criterion = nn.L1Loss()
    scaler = torch.cuda.amp.GradScaler()
    optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs - 1)
    
    val_loss_min = np.Inf
    
    for idx, epoch in enumerate(range(1, n_epochs + 1)):
        scheduler.step()
        model.to(device).train()
        train_loss = []

        print('Epoch: {:02d}/{:02d}'.format(epoch, n_epochs))
        print("TRAIN")

        loop = tqdm(train_loader)
        for X, y in loop:
            X = X.to(device).float()
            y = y.to(device).float()

            optimizer.zero_grad()

            with torch.cuda.amp.autocast():
                output = model(X)
                loss = criterion(output, y)
            
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss.append(loss.item())
            loop.set_description('current_loss: {:.5f}'.format(loss.item()))
            loop.set_postfix(loss=np.mean(train_loss))
        train_loss = np.mean(train_loss)
    
        model.eval()

        val_loss = []

        print("VAL")
        loop = tqdm(valid_loader)
        for X, y in loop:
            X = X.to(device).float()
            y = y.to(device).float()

            with torch.cuda.amp.autocast(), torch.no_grad():
                outputs = model(X)
                loss = criterion(outputs.float(), y)

            val_loss.append(loss.item())
            loop.set_description('current_loss: {:.5f}'.format(loss.item()))
            loop.set_postfix(loss=np.mean(val_loss))
        val_loss = np.mean(val_loss)
        
        wandb.log({"epoch": epoch, 
                "loss": train_loss, 
                "val_loss": val_loss,
                })
        
        if val_loss < val_loss_min:
            print('Valid loss improved from {:.5f} to {:.5f} saving model to {}'.format(val_loss_min, val_loss, CHECKPOINT))
            val_loss_min = val_loss
            torch.save(model.state_dict(), CHECKPOINT)
            artifact = wandb.Artifact(model_name, type='model')
            artifact.add_file(CHECKPOINT, name=f"fold{fold}_epoch{epoch}.pt")
            run.log_artifact(artifact)
        
    del model, optimizer
    gc.collect()
    torch.cuda.empty_cache()
    
    run.finish()

### Hyperparameters for BiGRU

In [ ]:
ckpt_dir = "."
model_name = "BiGRU"
project_name = f"G-Vent-{model_name}"
n_features = train_ft.shape[-1]
n_splits = 5
n_epochs = 10
init_lr = 1e-3
b_size = 512
num_workers = 16

### BiGRU Training

In [ ]:
train_ft.shape, targets.shape

In [ ]:
run.finish()

In [ ]:
def GVent_MAE(y, preds, u_out):
    """
    Metric for the problem
    """
    w = 1 - u_out
    
    assert y.shape == preds.shape and w.shape == y.shape, (y.shape, preds.shape, w.shape)
    
    mae = w * torch.abs(y - preds)
    mae = mae.sum() / w.sum()
    
    return mae

In [ ]:
kf = KFold(n_splits=n_splits, shuffle=True, random_state=2021)

for fold, (train_idx, test_idx) in enumerate(kf.split(train_ft, targets)):
    
    CHECKPOINT = '{}/{}_{}.pth'.format(ckpt_dir, model_name, fold)
    
    run = wandb.init(project=project_name, name=f"fold{fold}")
    
    print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
    X_train, X_valid = train_ft[train_idx], train_ft[test_idx]
    y_train, y_valid = targets[train_idx], targets[test_idx]
    
    train_ds = BiGRUDataset(X_train, mode="train", targets=y_train)
    valid_ds = BiGRUDataset(X_valid, mode="train", targets=y_valid)
    
    train_loader = DataLoader(train_ds, batch_size=b_size, sampler=RandomSampler(train_ds), num_workers=num_workers)
    valid_loader = DataLoader(valid_ds, batch_size=b_size, num_workers=num_workers)
    
    model = BiGRUModel(n_features=train_ft.shape[-1])
    
    criterion = nn.L1Loss()
    scaler = torch.cuda.amp.GradScaler()
    optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 1e-5, n_epochs - 1)
    
    val_loss_min = np.Inf
    
    for idx, epoch in enumerate(range(1, n_epochs + 1)):
        scheduler.step()
        model.to(device).train()
        train_loss = []

        print('Epoch: {:02d}/{:02d}'.format(epoch, n_epochs))
        print("TRAIN")

        loop = tqdm(train_loader)
        for X, y in loop:
            X = X.to(device).float()
            y = y.to(device).float()
            
            optimizer.zero_grad()

            with torch.cuda.amp.autocast():
                output = model(X)
                loss = criterion(output, y)
            
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss.append(loss.item())
            loop.set_description('current_loss: {:.5f}'.format(loss.item()))
            loop.set_postfix(loss=np.mean(train_loss))
        train_loss = np.mean(train_loss)
    
        model.eval()

        val_loss = []

        print("VAL")
        loop = tqdm(valid_loader)
        for X, y in loop:
            X = X.to(device).float()
            y = y.to(device).float()

            with torch.cuda.amp.autocast(), torch.no_grad():
                outputs = model(X)
                loss = criterion(outputs.float(), y)

            val_loss.append(loss.item())
            loop.set_description('current_loss: {:.5f}'.format(loss.item()))
            loop.set_postfix(loss=np.mean(val_loss))
        val_loss = np.mean(val_loss)
        
        wandb.log({"epoch": epoch, 
                "loss": train_loss, 
                "val_loss": val_loss,
                })
        
        if val_loss < val_loss_min:
            print('Valid loss improved from {:.5f} to {:.5f} saving model to {}'.format(val_loss_min, val_loss, CHECKPOINT))
            val_loss_min = val_loss
            torch.save(model.state_dict(), CHECKPOINT)
            artifact = wandb.Artifact(model_name, type='model')
            artifact.add_file(CHECKPOINT, name=f"fold{fold}_epoch{epoch}.pt")
            run.log_artifact(artifact)
        
    del model, optimizer
    gc.collect()
    torch.cuda.empty_cache()
    
    run.finish()

# Inference

In [ ]:
test_ft.shape, len(test_ids)

In [ ]:
test_ds = GeneralDataset(test_ft, mode="test", test_ids=test_ids)
test_loader = DataLoader(test_ds, batch_size=b_size, num_workers=0)

models = {}
model_paths = [r"../input/gventcnn-weights/fold0_epoch10",
               r"../input/gventcnn-weights/fold1_epoch10",
               r"../input/gventcnn-weights/fold2_epoch10",
               r"../input/gventcnn-weights/fold3_epoch10",
               r"../input/gventcnn-weights/fold4_epoch10",]

for fold, path in enumerate(model_paths):
    model = CNN()
    model.load_state_dict(torch.load(path))
    models[fold] = model

ens_preds = []
for fold, model in models.items():
    model.eval().to(device)
    
    ids_list = []
    preds = []
    
    loop = tqdm(test_loader)
    for ids, X in loop:
        X = X.to(device).float()
        ids_list.append(ids)

        with torch.no_grad():
            outputs = model(X)
            preds.append(outputs.data.cpu().numpy())
    
    preds = np.concatenate(preds).flatten()
    ens_preds.append(preds)

# Submission

In [ ]:
submission = pd.DataFrame({
    "id": np.concatenate(ids_list).flatten(),
    "pressure": np.mean(ens_preds, axis=0)
})

submission.to_csv("submission.csv", index=False)